# 函数与迭代器

本章介绍 Python 中函数、迭代器、生成器的使用，并介绍一些 Python 中函数：

- 高级函数：经常用到的 `lambda`，时而会使用的 `filter`，以及极少用到的 `reduce` 与 `map`
- 错误控制语句：`try .. except`

本章引入了下述 Python 库/模块。它们均为内置库/模块，无需额外的安装步骤：

In [1]:
import timeit
import cProfile
import tracemalloc, linecache

## 函数的使用

使用 `def` 定义函数。下例定义了一个名为 `func` 的函数。

- 参数：下面的 func 函数包含两个输入参数 `a`, `b` 。其中，参数 b 是一个可选参数，在无输入时会自动赋值为 0。
- 注释：Python 中建议（但不强制）标出函数的注释，一般用一对三个双引号来标记多行注释，作为函数的注释。
- 返回值：用 `return` 语句来返回一个对象。

In [2]:
def func(a, b=0):
    """
    This is a function that can meow.
    """
    return " ".join(["meow"] * (a + b))

### 函数参数

上例的函数接受两个参数：

In [3]:
s = func(a=1, b=2)
print(s)

meow meow meow


对应的参数名在传入时，可以省略。比如上例中的 `a=` 与 `b=` 可以省略：

In [4]:
func(1, 2)

'meow meow meow'

由于参数 b 有一个默认值 0，因此函数 func 也可以只传入一个参数（即参数 a）：

In [5]:
func(1)

'meow'

函数也接受以序列或者字典的方式传入参数，使用带星号 `*` 前缀的序列，或者带双星号 `**` 前缀的字典。

In [6]:
key_lst = [2]
func(1, *key_lst)

'meow meow meow'

In [7]:
key_dict = {"a": 1, "b": 2}
func(**key_dict)

'meow meow meow'

这种传参方式有时用在字符串的 `format()` 函数中：

In [8]:
key_dict = {"a": 1, "b": 2}
s = "{a} + {b}".format(**key_dict)
print(s)

1 + 2


### 函数帮助

用 `help()` 函数来查看函数的帮助，即函数开头的、以三个双引号括起的字符串：

In [9]:
help(func)

Help on function func in module __main__:

func(a, b=0)
    This is a function that can meow.



## 处理异常：try与raise

用 `try .. except` 语句来控制异常。比如在除法运算时，如果除数为0，那么会弹出异常：

In [10]:
def func(m, n):
    r = m / n
    return r

func(1, 0)

ZeroDivisionError: division by zero

Python 在除数为0是会弹出 ZeroDivisionError。常常见到的 Error 有：

| 异常名称 | 解释 | 引发示例 |
| ---: | :--- | ---: |
| | **-- 引用异常 --** |
| AttributeError | 引用不存在的属性。 | `int.split()` |
| IndexError | 引用不存在的索引 | `"abc"[3]` |
| KeyError | 引用不存在的字典键 | `{}["a"]` |
| NameError | 使用不存在的变量名 | / |
| | **-- 输入参数异常 --** |
| TypeError | 函数被应用在类型错误的对象上 | `abs("1")` |
| ValueError | 函数传入了类型允许但值不适合的参数 | `int("a")` |
| | **-- 其他 --** |
| AssertionError | 断言语句 assert 失败 | `assert(1 < 0)` |
| StopIteration | 迭代器结束迭代 | `next(iter(''))` |
| SyntaxError | 语法错误 | `a = ]` |
| KeyboardInterrupt | 用户从键盘终止了正运行的代码  | Ctrl+C 打断运行 |

要阅读完整的 Error 列表，请参考官方文档的[内置异常](https://docs.python.org/zh-cn/3/library/exceptions.html#exception-hierarchy)页面。

添加 `try .. except` 来处理 ZeroDivisionError：

In [11]:
def func(m, n):
    try:
        r = m / n
    except ZeroDivisionError:
        r = float('nan')
    return r

func(1, 0)

nan

还可以用 `except ERROR as ...` 并 print 来输出异常信息：

In [12]:
def func(m, n):
    try:
        r = m / n
    except ZeroDivisionError as e:
        r = float('nan')
        print(f"Error: {e}")
    return r

func(1, 0)

Error: division by zero


nan

其中的 except 语句可以处理多种 error 类型。可以：

- 把多个 error 组成一个元组，或者
- 使用多个 except 语句。

In [13]:
def func(m, n):
    try:
        r = m / n
    except (ZeroDivisionError, TypeError) as e:
        r = float('nan')
        print(f"Error: {e}")
    return r

func(1, 'x'), func(1, 0), func(1, 2)

Error: unsupported operand type(s) for /: 'int' and 'str'
Error: division by zero


(nan, nan, 0.5)

In [14]:
def func(m, n):
    try:
        r = m / n
    except ZeroDivisionError:
        r = float('nan')
    except TypeError:
        r = None
    return r

func(1, 'x'), func(1, 0), func(1, 2)

(None, nan, 0.5)

异常处理的最后一个 except 语句可以:

- 不接任何 error 类型（即 `except:` ），以表示处理所有未被之前 except 所处理的异常；或者
- 接受基类型 Exception（即 `except Exception as e:`），并输出异常信息 `e` 。

In [15]:
def func(m, n):
    try:
        r = m / n
    except ZeroDivisionError:
        r = float('nan')
    except Exception as e:
        r = None
        print(f'Unhandled error in func({m}, {n}): {e}')
    return r

x = [1, 2, 3]
func(1, 'x'), func(1, 0), func(1, 2)

Unhandled error in func(1, x): unsupported operand type(s) for /: 'int' and 'str'


(None, nan, 0.5)

最后，用户也可以用 `raise` 语句强制抛出异常：

In [16]:
raise(ValueError("This is a forced error."))

ValueError: This is a forced error.

## 迭代器

迭代器（iterator）是可用 `next()` 函数依次访问数据的一种数据对象。从用法上讲，它与序列的应用场合非常近似，但有性能优化方面的潜力。

迭代器可以用以下方式创建：

- 使用 iter() 函数强制转换一个序列对象
- 使用迭代器解析（或称生成器解析）
- 使用生成器，参考下方[生成器](#生成器)一节的内容
- 定义一个含有 `__next__()` 方法的类

In [17]:
s = "abc"
x = iter(s)

for _ in range(len(s)):
    print(next(x))

a
b
c


在被遍历一次后，迭代器的“指针”会抵达其末尾。如果继续访问，将会抛出 StopIteration 异常：

In [18]:
next(x)

StopIteration: 

顺带一提，在迭代器上应用 `next()` 实质上是调用了迭代器内部的 `__next__()` 方法：

In [19]:
x = iter("abc")
x.__next__()

'a'

通常，我们使用循环语句（而不是 `next()` 函数）来遍历迭代器：

In [20]:
x = iter("abc")
for c in x:
    print(c)

a
b
c


除了用 `iter()` 强制转换，迭代器也可以使用类似列表解析的“迭代器解析”，不过要将外侧的方括号换成圆括号：

In [21]:
x = (2 ** k for k in range(5))

for k, num in enumerate(x):
    print(f"2^{k}:\t{num}")

2^0:	1
2^1:	2
2^2:	4
2^3:	8
2^4:	16


### 迭代器的意义：性能优化

迭代器最显著的意义就在于性能优化。由于迭代器解析会依次以（内部的）`__next__()` 方法调用数据，因此在调用时才计算当前项的值，而不是在创建之时一次性地计算出每一项的值。这对于内存占用与优化控制流都较有意义。

下面是一个较极端的例子，模拟在重负担任务时使用列表与迭代器之间的差异。

In [22]:
# 2**(2**10) = 1.79769... x10^308
def func_lst():
    x = [2**(n) for n in range(2**15)]
    for i, k in enumerate(x):
        if k > 1.7e308:
            break
    print(i)

def func_iter():
    x = (2**(n) for n in range(2**15))
    for i, k in enumerate(x):
        if k > 1.7e308:
            break
    print(i)

本节中所用到的时间、内存性能测试的模块，都会在之后的标准库章节进行介绍。

### 时间开销对比

下面是用 timeit 模块对上述两个函数的运行时间测试结果。由于 `func_lst()` 函数创建了一个长为 $2\times 10^{15}$ 的列表，因此速度比 `func_iter()` 慢上不少。

In [23]:
# 运算一次所花费的时间
# -- 运行在我的古董级 CPU i7-6700HQ @2.60GHz 上。

time_lst = timeit.timeit(func_lst, number=1)
time_iter = timeit.timeit(func_iter, number=1)
print(f"List:\t{time_lst:.4f} sec\nIter:\t{time_iter:.4f} sec")

1024
1024
List:	1.6071 sec
Iter:	0.0011 sec


用 cProfile 模块查看 `func_lst()` 运算开销，可以看到用时最高的一项是 `<listcomp>`，即 list comprehesion 列表解析。解析这个长度巨大的列表占用了整个函数运行的绝大部分时间。函数 `func_iter()` 的总开销极短，故在这里省略。

顺便一提， cProfile 并不是一个基准测试工具（请使用 timeit），而主要用来查看各部分运行的耗时占比；它的时间度量可能并不准确。

In [24]:
cProfile.run('func_lst()')

1024
         40 function calls in 1.621 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    1.605    1.605 <ipython-input-22-eae10954e071>:2(func_lst)
        1    1.605    1.605    1.605    1.605 <ipython-input-22-eae10954e071>:3(<listcomp>)
        1    0.016    0.016    1.621    1.621 <string>:1(<module>)
        3    0.000    0.000    0.000    0.000 iostream.py:197(schedule)
        2    0.000    0.000    0.000    0.000 iostream.py:310(_is_master_process)
        2    0.000    0.000    0.000    0.000 iostream.py:323(_schedule_flush)
        2    0.000    0.000    0.000    0.000 iostream.py:386(write)
        3    0.000    0.000    0.000    0.000 iostream.py:93(_event_pipe)
        3    0.000    0.000    0.000    0.000 socket.py:342(send)
        3    0.000    0.000    0.000    0.000 threading.py:1017(_wait_for_tstate_lock)
        3    0.000    0.000    0.000    0.000 threading.py:1071(is_ali

### 内存开销对比

最后，我们用 tracemalloc 模块查看一下双方的内存占用，并打印占用内存最大的3条指令。

下例中的 `display_top` 函数引用自 Python 官方文档 [tracemalloc: Pretty Top](https://docs.python.org/zh-cn/3/library/tracemalloc.html#pretty-top)，仅改动了 limit 参数的默认值。

In [25]:
def display_top(snapshot, key_type='lineno', limit=3):
    snapshot = snapshot.filter_traces((
        tracemalloc.Filter(False, "<frozen importlib._bootstrap>"),
        tracemalloc.Filter(False, "<unknown>"),
    ))
    top_stats = snapshot.statistics(key_type)

    print("Top %s lines" % limit)
    for index, stat in enumerate(top_stats[:limit], 1):
        frame = stat.traceback[0]
        print("#%s: %s:%s: %.1f KiB"
              % (index, frame.filename, frame.lineno, stat.size / 1024))
        line = linecache.getline(frame.filename, frame.lineno).strip()
        if line:
            print('    %s' % line)

    other = top_stats[limit:]
    if other:
        size = sum(stat.size for stat in other)
        print("%s other: %.1f KiB" % (len(other), size / 1024))
    total = sum(stat.size for stat in top_stats)
    print("Total allocated size: %.1f KiB" % (total / 1024))

使用列表解析的 `func_lst()` 占用高达约 70 MB，而占用第二位的只有十几 KB。这说明绝大部分的内存使用都是创建这个巨大的列表的开销。

In [26]:
tracemalloc.start()

# func_lst()
x = [2**(n) for n in range(2**15)]
for i, k in enumerate(x):
    if k > 1.7e308:
        break

snapshot = tracemalloc.take_snapshot()
display_top(snapshot)

Top 3 lines
#1: <ipython-input-26-0957e906499f>:4: 71111.4 KiB
    x = [2**(n) for n in range(2**15)]
#2: d:\programming\python38\lib\site-packages\IPython\core\history.py:763: 0.4 KiB
    conn.execute("INSERT INTO history VALUES (?, ?, ?, ?)",
#3: d:\programming\python38\lib\codeop.py:136: 0.2 KiB
    codeob = compile(source, filename, symbol, self.flags, 1)
9 other: 0.8 KiB
Total allocated size: 71112.7 KiB


而使用迭代器解析的 `func_iter()` 的内存占用非常小，仅 200 余 KB。

In [27]:
tracemalloc.start()

# func_iter()
x = (2**(n) for n in range(2**15))
for i, k in enumerate(x):
    if k > 1.7e308:
        break

snapshot = tracemalloc.take_snapshot()
display_top(snapshot)

Top 3 lines
#1: d:\programming\python38\lib\linecache.py:137: 92.0 KiB
    lines = fp.readlines()
#2: d:\programming\python38\lib\site-packages\IPython\core\compilerop.py:101: 5.3 KiB
    return compile(source, filename, symbol, self.flags | PyCF_ONLY_AST, 1)
#3: d:\programming\python38\lib\json\decoder.py:353: 1.5 KiB
    obj, end = self.scan_once(s, idx)
108 other: 24.7 KiB
Total allocated size: 123.4 KiB


## 生成器

生成器（generator）是一种创建迭代器的简便工具。它用 `yield` 关键字来代替 `return`，使生成器在每次被访问时都返回一个值，以此实现“依次序取出数据”的迭代器效果。

In [28]:
def fprinting(seq):
    for i, val in enumerate(seq):
        yield f"{i}-th:\t{val}"

for k in fprinting("hello"):
    print(k)

0-th:	h
1-th:	e
2-th:	l
3-th:	l
4-th:	o


生成器解析（迭代器解析）在上文已经介绍过，可以减少内存使用。它可以直接作为序列参数传入函数（但不要忘记迭代器只能被遍历一次）；在下例中，生成器解析的结果被作为参数直接传给了 `sum()` 与 `str.join()` 函数。

In [29]:
n = sum(k**2 for k in range(5))
s = ' + '.join(f"{k**2}" for k in range(5))
print(s, "=", n)

0 + 1 + 4 + 9 + 16 = 30


## lambda 函数

`lambda` 称为匿名函数，它用来简洁地声明一个函数。比如下面的匿名函数 `f1` 与用 `def` 定义的函数 `f2` 作用相同：

In [30]:
f1 = lambda x,y: x+y
def f2(x, y):
    return x+y

f1(2,3) == f2(2,3)

True

## filter 函数

`filter` 称为过滤函数，将一个返回逻辑类型（True/False）的函数作为过滤器，用来过滤一个序列。序列中的经过函数能返回 True 的项会被保留下来，其余的项会被舍弃。

最后，被保留的项会以一个迭代器的形式返回。使用时请谨记迭代器的特性：迭代器在创建后只能被遍历一次。

In [31]:
f = lambda x: x > 0
lst = range(-3, 3)

filter(f, lst)

你可以将迭代器转为列表，或者直接在循环中处理：

In [32]:
x = filter(f, lst)  # 新建迭代器
print(list(x))

[1, 2]


In [33]:
x = filter(f, lst)  # 新建迭代器  
for k in x:
    print(k)

1
2


## reduce 函数\*

`reduce` 缩减函数将一个接受两个参数、返回一个值的函数应用在一个序列上，并依次以类似“累加”计算的方式来遍历整个序列。我认为这并不是一个常用的函数。

该函数在 Python 2 时代是一个可以直接调用的高级函数，但在 Python 3 版本需要从 functools 模块导入：

In [34]:
from functools import reduce

下面是一个 reduce 函数的例子：

In [35]:
# (((1+2)+3)+4) = 10.

d = reduce(lambda x,y: x+y, range(5))
print(d)

10


## map 函数\*

`map` 映射函数将一个函数应用在一个序列的每一项上，并返回一个迭代器。由于这个功能可以用迭代器解析的方法实现，因此它并不是一个常用的高级函数。

In [36]:
s = "123"
x = map(int, s)

for k in x:
    print(k)

1
2
3


它与迭代器解析的写法并没有显著的不同：

In [37]:
s = "123"
x = (int(k) for k in s)

for k in x:
    print(k)

1
2
3
